# Bedrock Knowledge Base via Your own Opensearch Database
If you have have any questions, please feel free to contact Hao Huang (tonyhh@amazon.com, GAIIC), Dong Xiaoqun(xiaoqunn@amazon.com, GenAI SSA).

- Step 0. Pre-request
- Step 1. Authentication
- Step 2. Build Amazon Opensearch Serveless Vector Database
- Step 3. Insert Knowledge to AOS
- Step 4. Create Bedrock Knowledge Base
    * Step 4.1. Create Bedrock Knowledge Base Role
    * Step 4.2. Update collection policy
- Step 5. Test knowledge bases
    * Step 5.1. Test knowledge bases retrive
    * Step 5.2. Test knowledge bases retrive and generate
    * Step 5.3. Associate to an agent and test

### Step 0. Pre-request

In [ ]:
!pip install opensearch-py
!pip install requests-aws4auth
!pip install boto3

### Step 1. Authentication

In [ ]:
import os
import json
import boto3
import botocore
import logging
import pprint
import time

from opensearchpy import OpenSearch, RequestsHttpConnection
from requests_aws4auth import AWS4Auth
from utils import createEncryptionPolicy, createNetworkPolicy, createCollection, waitForCollectionCreation

logging.basicConfig(
    format='[%(asctime)s] p%(process)s {%(filename)s:%(lineno)d} %(levelname)s - %(message)s',
    level=logging.INFO,
    encoding="utf-8"
)
logger = logging.getLogger(__name__)

service = 'aoss'
region = 'us-east-1'
credentials = boto3.Session().get_credentials()

awsauth = AWS4Auth(
    credentials.access_key,
    credentials.secret_key,
    region,
    service,
    session_token=credentials.token
)


### Step 2. Build Amazon Opensearch Serveless Vector Database


In [ ]:
# Notice: INDEX_NAME name need to start with "bedrock-knowledge-" for demo, we set all the policy below for "bedrock-knowledge-demo-". Or you can modify all the policy function for collection name.
client = boto3.client('opensearchserverless')
INDEX_NAME = "bedrock-knowledge-demo-invoice" 
Collection_Name = "bedrock-knowledge-demo-invoice"  

# Creates an encryption policy
createEncryptionPolicy(client, Collection_Name)

#Creates an network policy
createNetworkPolicy(client, Collection_Name)

In [ ]:

# Create collection access policy
"""
Notice: need add your own role/user arn in here, e.g.
"Principal":[ <br>
    "arn:aws:iam::{your-account-id}:user/XXX",  # credential role arn
    "arn:aws:iam::{your-account-id}:role/Admin"  # console
]
"""
policy = """
            [{
                \"Rules\":[
                    {
                        \"Resource\":[
                            \"index\/bedrock-knowledge-*\/*\"
                        ],
                        \"Permission\":[
                            \"aoss:CreateIndex\",
                            \"aoss:DeleteIndex\",
                            \"aoss:UpdateIndex\",
                            \"aoss:DescribeIndex\",
                            \"aoss:ReadDocument\",
                            \"aoss:WriteDocument\"
                        ],
                        \"ResourceType\": \"index\"
                    },
                    {
                        \"Resource\":[
                            \"collection\/bedrock-knowledge-*\"
                        ],
                        \"Permission\":[
                            \"aoss:CreateCollectionItems\",
                            \"aoss:DescribeCollectionItems\",
                            \"aoss:UpdateCollectionItems\"
                        ],
                        \"ResourceType\": \"collection\"
                    }
                ],
                \"Principal\":[
                    \"arn:aws:iam::{your-account-id}:user\/XXX\",
                    \"arn:aws:iam::{your-account-id}:role\/Admin\"
                ]
            }]
        """
    
try:
    response = client.create_access_policy(
        description='Data access policy for "bedrock-knowledge-" collections',
        name='bedrock-kb-demo-policy',
        policy=policy,
        type='data'
    )
    policyVersion = response["accessPolicyDetail"]["policyVersion"]
    print('\nAccess policy created:')
    print(response)
except botocore.exceptions.ClientError as error:
    if error.response['Error']['Code'] == 'ConflictException':
        print(
            '[ConflictException] An access policy with this name already exists.')
    else:
        raise error

In [ ]:
# Create collection
createCollection(client, Collection_Name)
# Waite for collection create completely
final_host, collectionarn = waitForCollectionCreation(client, Collection_Name)

In [ ]:
# Create index in collection

dimensions = 1536

client = OpenSearch(
        hosts=[{'host': final_host, 'port': 443}],
        http_auth=awsauth,
        use_ssl=True,
        verify_certs=True,
        connection_class=RequestsHttpConnection,
        timeout=300
    )

client.indices.create(
    INDEX_NAME,
    body={
            "settings":{
                "index":{
                "number_of_shards" : 1,
                "number_of_replicas" : 0,
                "knn": "true",
                "knn.algo_param.ef_search": 32
                }
            },
            "mappings":{
                "properties": {
                    "bedrock-knowledge-base-default-vector": {
                        "type": "knn_vector", 
                        "dimension": dimensions,
                        "method": {
                            "engine": "nmslib",
                            "space_type": "cosinesimil",
                            "name": "hnsw",
                            "parameters": {}
                        }
                    },
                    "AMAZON_BEDROCK_METADATA": {
                        "type": "text",
                        "index": False
                    },
                    "AMAZON_BEDROCK_TEXT_CHUNK": {
                        "type": "text"
                    },
                    "id": {
                    "type": "text",
                    "fields": {
                        "keyword": {
                            "type": "keyword",
                            "ignore_above": 256
                        }
                    }

                }
            }
        }
    }
)

### Step 3. Insert Knowledge to AOS

In [31]:
# Upload kb file to S3

bucket_name = "invoice-agent-demo-kb"
s3_client = boto3.client("s3", region)


file_name = "piaozone2.faq"
file_path = "./doc/"
response = s3_client.create_bucket(Bucket=bucket_name)
upload_file = s3_client.upload_file(os.path.join(file_path, file_name), bucket_name, file_name)
s3_path = f"s3://{bucket_name}/{file_name}"
s3_arn = f"arn:aws:s3:::{bucket_name}/{file_name}"

In [32]:
def create_vector_embedding_with_bedrock(text, s3_path, docs, embedding_modelId='amazon.titan-embed-text-v1'):
    brt = boto3.client(service_name='bedrock-runtime', region_name='us-east-1')
    body = json.dumps({
        "inputText": text
    })

    accept = 'application/json'
    contentType = 'application/json'
    response = brt.invoke_model(body=body, modelId=embedding_modelId, accept=accept, contentType=contentType)
    response_body = json.loads(response.get('body').read())
    embedding = response_body['embedding']
    info = {
        "AMAZON_BEDROCK_METADATA": '{{"source":"{}"}}'.format(s3_path), 
        "AMAZON_BEDROCK_TEXT_CHUNK": docs, 
        "bedrock-knowledge-base-default-vector": embedding, 
        "id": "0"
        }
    return info


# Use local file embedding text and insert to aos
# You can modify this function adapt to your docment format
def WriteToAos(file_path, index_name, s3_path):
    Q = ""
    A = ""
    with open(file_path, "r") as rf:
        for idx, line in enumerate(rf):
            if idx == 10:
                break
            if line.startswith("Question"):
                Q = line.split("Question:")[-1]
            elif line.startswith("Answer"):
                A = line.split("Answer:")[-1] 
            elif line.startswith("===="):
                doc_template = "Answer: {}"
                docs = doc_template.format(A)
                insert_body_q = create_vector_embedding_with_bedrock(Q, s3_path ,docs)
                insert_body_a = create_vector_embedding_with_bedrock(A, s3_path, docs)

                response = client.index(
                    index=index_name,
                    body=insert_body_q,
                )
                response = client.index(
                    index=index_name,
                    body=insert_body_a,
                )
                print(f'Document added: {idx}')
                print(response)
                Q = ""
                A = ""
            else:
                continue

    if Q != "" and A != "":
        doc_template = "Answer: {}"
        docs = doc_template.format(A)
        insert_body_q = create_vector_embedding_with_bedrock(Q, s3_path ,docs)
        insert_body_a = create_vector_embedding_with_bedrock(A, s3_path, docs)

        response = client.index(
            index=index_name,
            body=insert_body_q,
        )
        response = client.index(
            index=index_name,
            body=insert_body_a,
        )
        print('\nDocument added:')
        print(response)  

In [ ]:
WriteToAos("./doc/piaozone2.faq", INDEX_NAME, s3_path)

### Step 4. Create Bedrock Knowledge Base

#### Step 4.1. Create Bedrock Knowledge Base Role

In [34]:
from utils import create_role, create_policy, create_policy, attach_policy

In [ ]:
# Create Bedrock Knowledge Base role

# Role name must startwith "AmazonBedrockExecutionRoleForKnowledgeBase_" 
iam = boto3.resource("iam")
bedrock_knowledge_base_role_name = "AmazonBedrockExecutionRoleForKnowledgeBase_demo"
bedrock_knowledge_base_role = create_role(
    iam,
    bedrock_knowledge_base_role_name,
    ["bedrock.amazonaws.com"]
)
print(bedrock_knowledge_base_role.arn)

In [ ]:
# Create s3, bedrock invoke, aos policy
s3_file_policy = create_policy(
    iam,
    "invoice-kb-s3-demo-policy",
    "Policy for IAM demonstration.",
    ["s3:GetObject","s3:ListBucket"],
    s3_arn
)

bedrock_kb_invoke_demo_policy = create_policy(
    iam,
    "invoice-kb-invoke-demo-policy",
    "Policy for knowleadge Base demonstration.",
    "bedrock:InvokeModel",
    "arn:aws:bedrock:us-east-1::foundation-model/amazon.titan-embed-text-v1"
)


bedrock_kb_os_demo_policy = create_policy(
    iam,
    "invoice-kb-aos-demo-policy",
    "Policy for knowleadge Base demonstration.",
    "aoss:APIAccessAll",
    collectionarn 
)

In [ ]:
# attach policy to Bedrock Agent role
attach_policy(
    iam,
    bedrock_knowledge_base_role_name,
    s3_file_policy.arn
) 
attach_policy(
    iam,
    bedrock_knowledge_base_role_name,
    bedrock_kb_invoke_demo_policy.arn
)
attach_policy(
    iam,
    bedrock_knowledge_base_role_name,
    bedrock_kb_os_demo_policy.arn
)

#### Step 4.2. Update collection policy

In [ ]:
client.list_access_policies(type="data")

In [ ]:
client = boto3.client('opensearchserverless')
"""
Add arn of the bedrock knowledge base role which you created in Principal, You need replace {your-account-id} by your own ID
e.g.
"Principal":[
    "arn:aws:iam::{your-account-id}:role/AmazonBedrockExecutionRoleForKnowledgeBase_demo"
    "arn:aws:iam::{your-account-id}:user/XXX", 
    "arn:aws:iam::{your-account-id}:role/Admin"
]

"""


policy = """
            [{
                \"Rules\":[
                    {
                        \"Resource\":[
                            \"index\/bedrock-knowledge-*\/*\"
                        ],
                        \"Permission\":[
                            \"aoss:CreateIndex\",
                            \"aoss:DeleteIndex\",
                            \"aoss:UpdateIndex\",
                            \"aoss:DescribeIndex\",
                            \"aoss:ReadDocument\",
                            \"aoss:WriteDocument\"
                        ],
                        \"ResourceType\": \"index\"
                    },
                    {
                        \"Resource\":[
                            \"collection\/bedrock-knowledge-*\"
                        ],
                        \"Permission\":[
                            \"aoss:CreateCollectionItems\",
                            \"aoss:DescribeCollectionItems\",
                            \"aoss:UpdateCollectionItems\"
                        ],
                        \"ResourceType\": \"collection\"
                    }
                ],
                \"Principal\":[
                    \"arn:aws:iam::{your-account-id}:role\/AmazonBedrockExecutionRoleForKnowledgeBase_demo\",
                    \"arn:aws:iam::{your-account-id}:user\/XXX\",
                    \"arn:aws:iam::{your-account-id}:role\/Admin\"
                ]
            }]
        """
    
try:
    response = client.update_access_policy(
        description='Data access policy for bedrock-knowledge- collections',
        name='bedrock-kb-demo-policy',
        policy=policy,
        type='data',
        policyVersion=policyVersion
    )
    print('\nAccess policy created:')
    print(response)
except botocore.exceptions.ClientError as error:
    if error.response['Error']['Code'] == 'ConflictException':
        print(
            '[ConflictException] An access policy with this name already exists.')
    else:
        raise error

In [56]:
# Now, create bedorck Konowledge base

client = boto3.client("bedrock-agent", region_name=region)

response = client.create_knowledge_base(
    name='invoice-kb-demo',
    description='invocie demo notebook-test',
    roleArn=bedrock_knowledge_base_role.arn,
    knowledgeBaseConfiguration={
        'type': 'VECTOR',
        'vectorKnowledgeBaseConfiguration': {
            'embeddingModelArn': 'arn:aws:bedrock:us-east-1::foundation-model/amazon.titan-embed-text-v1'
        }
    },
    storageConfiguration={
        'type': 'OPENSEARCH_SERVERLESS',
        'opensearchServerlessConfiguration': {
            'collectionArn': collectionarn,
            'vectorIndexName': INDEX_NAME,
            'fieldMapping': {
                'vectorField': 'bedrock-knowledge-base-default-vector',
                'textField': 'AMAZON_BEDROCK_TEXT_CHUNK',
                'metadataField': 'AMAZON_BEDROCK_METADATA'
            }
        }
    },
)

In [58]:
knowledge_base_id = response['knowledgeBase']['knowledgeBaseId']
knowledge_base_arn = response["knowledgeBase"]["knowledgeBaseArn"]
print(knowledge_base_id)
print(knowledge_base_arn)

QN5VICEX1H


### Step 5. Test knowledge bases

#### Step 5.1. Test knowledge bases retrive

In [62]:
knowledgebases_client = boto3.client("bedrock-agent-runtime", region)
response = knowledgebases_client.retrieve(
    knowledgeBaseId=knowledge_base_id,
    retrievalQuery={
        'text': '使用单位线上申请是否一定和线下资料申请时使用单位 保持一致?如果想添加使用单位后续如何添加申请 ?'
    },
    retrievalConfiguration={
        'vectorSearchConfiguration': {
            'numberOfResults': 1 
        }
    },
)
pprint.pprint(response)

{'ResponseMetadata': {'HTTPHeaders': {'connection': 'keep-alive',
                                      'content-length': '405',
                                      'content-type': 'application/json',
                                      'date': 'Wed, 13 Dec 2023 08:16:43 GMT',
                                      'x-amzn-requestid': '514c7388-85a9-43f6-8ac0-35f1e9c389b0'},
                      'HTTPStatusCode': 200,
                      'RequestId': '514c7388-85a9-43f6-8ac0-35f1e9c389b0',
                      'RetryAttempts': 0},
 'retrievalResults': [{'content': {'text': 'Answer:  接入单位完成乐企直连后 ，如需新增使用单位 '
                                           '，只需在乐企平台邀请使用单位 ，填写纳税人信息 '
                                           '，新增使用单位主管税务 机关审批通过后即可使用乐企能力 '
                                           '，无需线下再次申请 。\n'},
                       'location': {'s3Location': {'uri': 's3://invoice-agent-demo-kb/piaozone2.faq'},
                                    'type': 'S3'},
                    

#### Step 5.2. Test knowledge bases retrive and generate

In [65]:
response = knowledgebases_client.retrieve_and_generate(
    input={
        'text': '使用单位线上申请是否一定和线下资料申请时使用单位 保持一致?如果想添加使用单位后续如何添加申请 ?'
    },
    
    retrieveAndGenerateConfiguration={
        'type': 'KNOWLEDGE_BASE',
        'knowledgeBaseConfiguration': {
            'knowledgeBaseId': knowledge_base_id,
            'modelArn': 'arn:aws:bedrock:us-east-1::foundation-model/anthropic.claude-v2'
        }
    },
)
pprint.pprint(response)

{'ResponseMetadata': {'HTTPHeaders': {'connection': 'keep-alive',
                                      'content-length': '1605',
                                      'content-type': 'application/json',
                                      'date': 'Wed, 13 Dec 2023 08:17:37 GMT',
                                      'x-amzn-requestid': '55ee4252-b294-481b-9b3e-1c004764cfd0'},
                      'HTTPStatusCode': 200,
                      'RequestId': '55ee4252-b294-481b-9b3e-1c004764cfd0',
                      'RetryAttempts': 0},
 'citations': [{'generatedResponsePart': {'textResponsePart': {'span': {'end': 117,
                                                                        'start': 0},
                                                               'text': '根据搜索结果3和4,使用单位线上申请与线下资料申请时使用单位不需要一定保持一致。接入单位完成乐企直连后,如需新增使用单位,只需在乐企平台邀请使用单位,填写纳税人信息,新增使用单位主管税务机关审批通过后即可使用乐企能力,无需线下再次申请。'}},
                'retrievedReferences': [{'content': {'text': 'Answer:  '
                  

#### Step 5.3. Associate to an agent and test

In [82]:
# Associate to an agent
agent_id = "FCZI2TH2IY"
response = client.associate_agent_knowledge_base(
    agentId=agent_id,
    agentVersion='DRAFT',
    description='Use this knowledge base whenever user ask information for issue an invoice',
    knowledgeBaseId=knowledge_base_id
)
pprint.pprint(response)

{'ResponseMetadata': {'HTTPHeaders': {'connection': 'keep-alive',
                                      'content-length': '267',
                                      'content-type': 'application/json',
                                      'date': 'Wed, 13 Dec 2023 08:54:19 GMT',
                                      'x-amz-apigw-id': 'P36hSFWHoAMEMnw=',
                                      'x-amzn-requestid': '034e69cf-65f3-429b-be95-ba670faaf485',
                                      'x-amzn-trace-id': 'Root=1-6579713b-0fa2eda204167b7d5e5283ce'},
                      'HTTPStatusCode': 200,
                      'RequestId': '034e69cf-65f3-429b-be95-ba670faaf485',
                      'RetryAttempts': 0},
 'agentKnowledgeBase': {'createdAt': datetime.datetime(2023, 12, 13, 8, 54, 19, 288694, tzinfo=tzutc()),
                        'description': 'Use this knowledge base whenever user '
                                       'ask information for issue an invoice',
               

In [83]:
agent_alias_name = "demo_agent_kb_test"
response = client.prepare_agent(agentId=agent_id)

# Need wait for preparing
time.sleep(15)

agent_alias_description = "add kb version"
agent_alias = client.create_agent_alias(
    agentId=agent_id,
    agentAliasName=agent_alias_name,
    description=agent_alias_description
)
agent_alias_id = agent_alias['agentAlias']['agentAliasId']
# Need change to "bedrock-agent-runtime" 
client = boto3.client("bedrock-agent-runtime", region_name=region)

def invoke(question: str, sessionid: str, agent_id: str, agent_alias_id: str, enable_trace=False):
    final_answer = ""
    response = client.invoke_agent(inputText=question,
        agentId=agent_id,
        agentAliasId=agent_alias_id,
        sessionId=sessionid,
        enableTrace=enable_trace
    )
    event_stream = response['completion']
    try:
        for event in event_stream:        
            # print(event)
            if 'chunk' in event:
                data = event['chunk']['bytes']
                final_answer = data.decode('utf8')
                print(f"Final answer ->\n{final_answer}") 
                end_event_received = True
                # End event indicates that the request finished successfully
            elif 'trace' in event:
                logger.info(json.dumps(event['trace'], indent=2))
            else:
                raise Exception("unexpected event.", event)
    except Exception as e:
        raise Exception("unexpected event.", e)
    # return final_answer 

In [84]:
import uuid
sessionid = str(uuid.uuid1())
enable_trace:bool = True

In [85]:
question = "你好"
answer = invoke(question, sessionid, agent_id, agent_alias_id, enable_trace)

[2023-12-13 16:56:09,677] p13994 {2047479949.py:36} INFO - {
  "agentId": "FCZI2TH2IY",
  "agentAliasId": "BTDLGUORHL",
  "sessionId": "5dfd84c4-9995-11ee-acf6-4e80c71e97ec",
  "trace": {
    "preProcessingTrace": {
      "modelInvocationInput": {
        "traceId": "10724582-8cf9-4ee0-aee5-f382fba98462-pre-0",
        "text": "\n\nHuman: You are a classifying agent that filters user inputs into categories. Your job is to sort these inputs before they are passed along to our function calling agent. The purpose of our function calling agent is to call functions in order to answer user's questions.\n\nHere is the list of functions we are providing to our function calling agent. The agent is not allowed to call any other functions beside the ones listed here:\n<functions>\n<function>\n<function_name>POST::InvoiceService_demo::generatePreviewInvoiceImage</function_name>\n<function_description>Generate a temporary preview invoice image.</function_description>\n<required_argument>buyer_tax_n

Final answer ->
您好,我是一个发票助手。


In [86]:
question = "我想开发票"
answer = invoke(question, sessionid, agent_id, agent_alias_id, enable_trace)

[2023-12-13 16:56:53,972] p13994 {2047479949.py:36} INFO - {
  "agentId": "FCZI2TH2IY",
  "agentAliasId": "BTDLGUORHL",
  "sessionId": "5dfd84c4-9995-11ee-acf6-4e80c71e97ec",
  "trace": {
    "preProcessingTrace": {
      "modelInvocationInput": {
        "traceId": "f32d57a8-b875-4ab4-bcaa-da49dbda726e-pre-0",
        "text": "\n\nHuman: You are a classifying agent that filters user inputs into categories. Your job is to sort these inputs before they are passed along to our function calling agent. The purpose of our function calling agent is to call functions in order to answer user's questions.\n\nHere is the list of functions we are providing to our function calling agent. The agent is not allowed to call any other functions beside the ones listed here:\n<functions>\n<function>\n<function_name>POST::InvoiceService_demo::generatePreviewInvoiceImage</function_name>\n<function_description>Generate a temporary preview invoice image.</function_description>\n<required_argument>buyer_tax_n

Final answer ->
您好,请提供以下信息以便我生成发票:
1. 买方纳税人识别号
2. 买方公司名称  
3. 用户ID
4. 产品详细信息(名称、编码、金额)


In [88]:
question = "我想查询开发票相关知识，直连单位是指全集团的年收入吗？"
answer = invoke(question, sessionid, agent_id, agent_alias_id, enable_trace)

[2023-12-13 17:04:43,947] p13994 {2047479949.py:36} INFO - {
  "agentId": "FCZI2TH2IY",
  "agentAliasId": "BTDLGUORHL",
  "sessionId": "5dfd84c4-9995-11ee-acf6-4e80c71e97ec",
  "trace": {
    "preProcessingTrace": {
      "modelInvocationInput": {
        "traceId": "38e54662-ebfe-488a-9526-e401988238c5-pre-0",
        "text": "\n\nHuman: You are a classifying agent that filters user inputs into categories. Your job is to sort these inputs before they are passed along to our function calling agent. The purpose of our function calling agent is to call functions in order to answer user's questions.\n\nHere is the list of functions we are providing to our function calling agent. The agent is not allowed to call any other functions beside the ones listed here:\n<functions>\n<function>\n<function_name>POST::InvoiceService_demo::generatePreviewInvoiceImage</function_name>\n<function_description>Generate a temporary preview invoice image.</function_description>\n<required_argument>buyer_tax_n

Exception: ('unexpected event.', EventStreamError('An error occurred (accessDeniedException) when calling the InvokeAgent operation: User: arn:aws:sts::596899493901:assumed-role/AmazonBedrockExecutionRoleForAgents_demo/BedrockAgentRetrieveSession-QN5VICEX1H is not authorized to perform: bedrock:Retrieve on resource: arn:aws:bedrock:us-east-1:596899493901:knowledge-base/QN5VICEX1H because no identity-based policy allows the bedrock:Retrieve action'))

In [ ]:
# If you did'n grant knowledge base retrieve permission to bedrock agent role
knowledge_base_arn = "arn:aws:bedrock:us-east-1:596899493901:knowledge-base/*"

kb_retrive_policy = create_policy(
    iam,
    "invoice-agent-kb-demo-policy",
    "Policy for agent kb retreive.",
    ["bedrock:Retrieve"],
    [knowledge_base_arn] 
)

attach_policy(
    iam,
    "AmazonBedrockExecutionRoleForAgents_demo",
    kb_retrive_policy.arn 
)

In [95]:
question = "我想查询开发票相关知识，直连单位是指全集团的年收入吗？"
answer = invoke(question, sessionid, agent_id, agent_alias_id, enable_trace)

[2023-12-13 17:30:06,862] p13994 {2047479949.py:36} INFO - {
  "agentId": "FCZI2TH2IY",
  "agentAliasId": "BTDLGUORHL",
  "sessionId": "5dfd84c4-9995-11ee-acf6-4e80c71e97ec",
  "trace": {
    "preProcessingTrace": {
      "modelInvocationInput": {
        "traceId": "85ab8060-982b-4359-9245-7764ec94d355-pre-0",
        "text": "\n\nHuman: You are a classifying agent that filters user inputs into categories. Your job is to sort these inputs before they are passed along to our function calling agent. The purpose of our function calling agent is to call functions in order to answer user's questions.\n\nHere is the list of functions we are providing to our function calling agent. The agent is not allowed to call any other functions beside the ones listed here:\n<functions>\n<function>\n<function_name>POST::InvoiceService_demo::generatePreviewInvoiceImage</function_name>\n<function_description>Generate a temporary preview invoice image.</function_description>\n<required_argument>buyer_tax_n

Final answer ->
根据知识库的搜索结果,直连单位是指全集团的年收入,而不是单个纳税人识别号的年收入。
